The methodology was taken from  : https://nlpiation.medium.com/is-it-possible-to-do-sentiment-analysis-on-unlabeled-data-using-bert-feat-vader-experiment-357bba53768c

# importing and installing packages


In [ ]:
from nltk.tokenize import word_tokenize, RegexpTokenizer
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split  # Import train_test_split function
import nltk
nltk.download("vader_lexicon")
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, BertTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from google.colab import drive
drive.mount('/content/gdrive')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


Mounted at /content/gdrive


In [ ]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.0 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade transformers
!pip install --upgrade simpletransformers


In [ ]:
!pip install --upgrade transformers


In [ ]:
pip install git+https://github.com/huggingface/transformers.git


# Reading the dataframe

In [ ]:
df = pd.read_csv("/content/gdrive/MyDrive/bert_result/KpopMusic_Labelled.csv", index_col = [0])
temp2 = df

In [ ]:
# Define the convert_label function
def convert_label(inp):
    if inp == 'Positive':
        return 1
    elif inp == 'Negative':
        return 0
    else:
        return None

# Filter out rows with 'Neutral' sentiment and then apply the conversion to the remaining rows.
temp2 = temp2[temp2["sentiment"] != 'Neutral']
temp2.loc[:, "sentiment"] = temp2["sentiment"].apply(lambda x: convert_label(x))


## Convert the sentiment label in the dataframe from {1, -1} to {1, 0}

In [ ]:
def convert_label(inp):
    return 0 if inp == 'Negative' else 1

temp2["sentiment"] = temp2["sentiment"].apply(lambda x: convert_label(x))

In [ ]:
temp2['sentiment'].value_counts()

1    637
Name: sentiment, dtype: int64

In [ ]:
test_df = temp2[['LyricsClean', 'sentiment']]

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.sentiment import SentimentIntensityAnalyzer

# Load the data from the CSV file
df = pd.read_csv("/content/gdrive/MyDrive/bert_result/KpopMusic_Labelled.csv", index_col=[0])

# Define the convert_label function to convert sentiments to numeric values
def convert_label(inp):
    return 0 if inp == 'Negative' else 1

# Filter out rows with 'Neutral' sentiment and then apply the conversion to the remaining rows.
df = df[df["sentiment"] != 'Neutral']
df["sentiment"] = df["sentiment"].apply(lambda x: convert_label(x))

# Split the data into training, validation, and test sets
train, test = train_test_split(df, test_size=0.2, random_state=8)
train, val = train_test_split(train, test_size=0.25, random_state=8)

# Sentiment Analysis using VADER
analyzer = SentimentIntensityAnalyzer()

def vader_sentiment_result(sent):
    scores = analyzer.polarity_scores(sent)

    if scores["neg"] > scores["pos"]:
        return 0

    return 1

# Convert "LyricsClean" column to string type
train["LyricsClean"] = train["LyricsClean"].astype(str)
val["LyricsClean"] = val["LyricsClean"].astype(str)
test["LyricsClean"] = test["LyricsClean"].astype(str)  # Add this line to convert the test set as well

# Apply sentiment analysis function
train["vader_result"] = train["LyricsClean"].apply(lambda x: vader_sentiment_result(x))
val["vader_result"] = val["LyricsClean"].apply(lambda x: vader_sentiment_result(x))
test["vader_result"] = test["LyricsClean"].apply(lambda x: vader_sentiment_result(x))  # Add this line for the test set

print(train.shape)
print(val.shape)
print(test.shape)


(381, 14)
(128, 14)
(128, 14)


## Split the dataset into training, validation and test set

In [ ]:
train, test = train_test_split(test_df, test_size=0.2, shuffle = True, random_state = 8)
train, val = train_test_split(train, test_size=0.25, random_state= 8)

In [ ]:
train.shape

(381, 2)

In [ ]:
val.shape

(128, 2)

In [ ]:
test.shape

(128, 2)

# VADER analysis

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

def vader_sentiment_result(sent):
    scores = analyzer.polarity_scores(sent)

    if scores["neg"] > scores["pos"]:
        return 0

    return 1

# Convert "LyricsClean" column to string type
train["LyricsClean"] = train["LyricsClean"].astype(str)
val["LyricsClean"] = val["LyricsClean"].astype(str)

# Apply sentiment analysis function
train["vader_result"] = train["LyricsClean"].apply(lambda x: vader_sentiment_result(x))
val["vader_result"] = val["LyricsClean"].apply(lambda x: vader_sentiment_result(x))


# BERT

## Training

Please note that training the BERT model will take a long time. Training of even 5 epochs took around 2-3 hours.

In [ ]:
!pip install transformers[torch]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 5.0 MB/s eta 0:00:00


In [ ]:
!pip install transformers==4.8.2
!pip install torch==1.9.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 29.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 17.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895239 sha256=ca4527986d5848491b6d7e78b15759572b79b190e8a81842203ba07654c82ab7
  Stored in directory: /root/.cache/pip/wheels/00/24/97/a2ea5324f36bc626e1ea0267f33db6aa80d157ee977e9e42fb
Successfully buil

In [ ]:
import os
os.environ["USE_ACCELERATE"] = "0"

from nltk.tokenize import word_tokenize, RegexpTokenizer
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import nltk
nltk.download("vader_lexicon")
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, BertTokenizer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


epoch 5

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Initialize the tokenizer and model for BERT
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")
model = BertForSequenceClassification.from_pretrained("bert-large-uncased")

# The dataset class
class TheDataset(Dataset):
    def __init__(self, reviews, sentiments, tokenizer):
        self.reviews = reviews
        self.sentiments = sentiments
        self.tokenizer = tokenizer
        self.max_len = tokenizer.model_max_length

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, index):
        review = str(self.reviews[index])
        sentiments = self.sentiments[index]

        encoded_review = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
            padding="max_length",
            truncation=True
        )

        return {
            'input_ids': encoded_review['input_ids'][0],
            'attention_mask': encoded_review['attention_mask'][0],
            'labels': torch.tensor(sentiments, dtype=torch.long)
        }

# Prepare the Train/Validation sets
train_dataset = TheDataset(
    reviews=train.LyricsClean.tolist(),
    sentiments=train.sentiment.tolist(),
    tokenizer=tokenizer,
)

val_dataset = TheDataset(
    reviews=val.LyricsClean.tolist(),
    sentiments=val.sentiment.tolist(),
    tokenizer=tokenizer,
)

# Freeze BERT except (the 24th layer + the last pooler layer)
for name, param in model.bert.named_parameters():
    if (not name.startswith('pooler')) and "layer.23" not in name:
        param.requires_grad = False

# Define the function to get the accuracy
def compute_metrics(preds, labels):
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Define the training parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
train_batch_size = 16
eval_batch_size = 64
num_train_epochs = 5
weight_decay = 0.01
warmup_steps = 500
learning_rate = 2e-5

# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=eval_batch_size)

# Create optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
total_steps = len(train_dataloader) * num_train_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

# Start pre-training!
model.train()
for epoch in range(num_train_epochs):
    print(f"Epoch {epoch + 1}/{num_train_epochs}")
    total_loss = 0.0
    total_steps = len(train_dataloader)

    # Initialize variables to store metrics for the epoch
    epoch_preds = []
    epoch_labels = []

    for step, batch in enumerate(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        # Get predictions and labels to compute metrics
        logits = outputs.logits
        preds = logits.argmax(dim=1)
        labels = batch['labels']

        # Store predictions and labels for the epoch
        epoch_preds.extend(preds.detach().cpu().numpy())
        epoch_labels.extend(labels.detach().cpu().numpy())

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        if step % 50 == 0:
            print(f"Step {step}/{total_steps}, Loss: {loss.item()}")

    # Calculate average loss for the epoch
    avg_loss = total_loss / total_steps
    print(f"Avg Loss for Epoch {epoch + 1}: {avg_loss}")

    # Calculate metrics for the epoch
    metrics = compute_metrics(preds=epoch_preds, labels=epoch_labels)
    print(f"Epoch {epoch + 1} Metrics:")
    print("Accuracy: ", metrics['accuracy'])
    print("Precision: ", metrics['precision'])
    print("Recall: ", metrics['recall'])
    print("F1 Score: ", metrics['f1'])

    print("Epoch completed.")

    # Now, let's validate the model after each epoch
    model.eval()
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for val_batch in val_dataloader:
            val_batch = {k: v.to(device) for k, v in val_batch.items()}
            val_outputs = model(**val_batch)
            val_logits = val_outputs.logits
            val_preds.extend(val_logits.argmax(dim=1).detach().cpu().numpy())
            val_labels.extend(val_batch['labels'].detach().cpu().numpy())

    val_metrics = compute_metrics(preds=val_preds, labels=val_labels)
    print("Validation Metrics:")
    print("Accuracy: ", val_metrics['accuracy'])
    print("Precision: ", val_metrics['precision'])
    print("Recall: ", val_metrics['recall'])
    print("F1 Score: ", val_metrics['f1'])

    # Set the model back to training mode
    model.train()

# Save the model
model.save_pretrained("/content/gdrive/MyDrive/bert_result/epoch 5")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
Step 0/24, Loss: 1.080095887184143
Avg Loss for Epoch 1: 0.9703689490755399
Epoch 1 Metrics:
Accuracy:  0.2230971128608924
Precision:  0.75
Recall:  0.02
F1 Score:  0.03896103896103896
Epoch completed.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Metrics:
Accuracy:  0.1484375
Precision:  0.0
Recall:  0.0
F1 Score:  0.0
Epoch 2/5
Step 0/24, Loss: 0.9945811033248901
Avg Loss for Epoch 2: 0.9410428057114283
Epoch 2 Metrics:
Accuracy:  0.23097112860892388
Precision:  0.7333333333333333
Recall:  0.03666666666666667
F1 Score:  0.06984126984126984
Epoch completed.
Validation Metrics:
Accuracy:  0.15625
Precision:  1.0
Recall:  0.009174311926605505
F1 Score:  0.018181818181818184
Epoch 3/5
Step 0/24, Loss: 0.8799847364425659
Avg Loss for Epoch 3: 0.842610277235508
Epoch 3 Metrics:
Accuracy:  0.28346456692913385
Precision:  0.7647058823529411
Recall:  0.13
F1 Score:  0.22222222222222224
Epoch completed.
Validation Metrics:
Accuracy:  0.3203125
Precision:  0.8928571428571429
Recall:  0.22935779816513763
F1 Score:  0.36496350364963503
Epoch 4/5
Step 0/24, Loss: 0.7563066482543945
Avg Loss for Epoch 4: 0.7132726137836775
Epoch 4 Metrics:
Accuracy:  0.5170603674540682
Precision:  0.8222222222222222
Recall:  0.49333333333333335
F1

epoch 10

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Initialize the tokenizer and model for BERT
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")
model = BertForSequenceClassification.from_pretrained("bert-large-uncased")

# The dataset class
class TheDataset(Dataset):
    def __init__(self, reviews, sentiments, tokenizer):
        self.reviews = reviews
        self.sentiments = sentiments
        self.tokenizer = tokenizer
        self.max_len = tokenizer.model_max_length

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, index):
        review = str(self.reviews[index])
        sentiments = self.sentiments[index]

        encoded_review = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
            padding="max_length",
            truncation=True
        )

        return {
            'input_ids': encoded_review['input_ids'][0],
            'attention_mask': encoded_review['attention_mask'][0],
            'labels': torch.tensor(sentiments, dtype=torch.long)
        }

# Prepare the Train/Validation sets
train_dataset = TheDataset(
    reviews=train.LyricsClean.tolist(),
    sentiments=train.sentiment.tolist(),
    tokenizer=tokenizer,
)

val_dataset = TheDataset(
    reviews=val.LyricsClean.tolist(),
    sentiments=val.sentiment.tolist(),
    tokenizer=tokenizer,
)

# Freeze BERT except (the 24th layer + the last pooler layer)
for name, param in model.bert.named_parameters():
    if (not name.startswith('pooler')) and "layer.23" not in name:
        param.requires_grad = False

# Define the function to get the accuracy
def compute_metrics(preds, labels):
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Define the training parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
train_batch_size = 16
eval_batch_size = 64
num_train_epochs = 10
weight_decay = 0.01
warmup_steps = 500
learning_rate = 2e-5

# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=eval_batch_size)

# Create optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
total_steps = len(train_dataloader) * num_train_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

# Start pre-training!
model.train()
for epoch in range(num_train_epochs):
    print(f"Epoch {epoch + 1}/{num_train_epochs}")
    total_loss = 0.0
    total_steps = len(train_dataloader)

    # Initialize variables to store metrics for the epoch
    epoch_preds = []
    epoch_labels = []

    for step, batch in enumerate(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        # Get predictions and labels to compute metrics
        logits = outputs.logits
        preds = logits.argmax(dim=1)
        labels = batch['labels']

        # Store predictions and labels for the epoch
        epoch_preds.extend(preds.detach().cpu().numpy())
        epoch_labels.extend(labels.detach().cpu().numpy())

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        if step % 50 == 0:
            print(f"Step {step}/{total_steps}, Loss: {loss.item()}")

    # Calculate average loss for the epoch
    avg_loss = total_loss / total_steps
    print(f"Avg Loss for Epoch {epoch + 1}: {avg_loss}")

    # Calculate metrics for the epoch
    metrics = compute_metrics(preds=epoch_preds, labels=epoch_labels)
    print(f"Epoch {epoch + 1} Metrics:")
    print("Accuracy: ", metrics['accuracy'])
    print("Precision: ", metrics['precision'])
    print("Recall: ", metrics['recall'])
    print("F1 Score: ", metrics['f1'])

    print("Epoch completed.")

    # Now, let's validate the model after each epoch
    model.eval()
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for val_batch in val_dataloader:
            val_batch = {k: v.to(device) for k, v in val_batch.items()}
            val_outputs = model(**val_batch)
            val_logits = val_outputs.logits
            val_preds.extend(val_logits.argmax(dim=1).detach().cpu().numpy())
            val_labels.extend(val_batch['labels'].detach().cpu().numpy())

    val_metrics = compute_metrics(preds=val_preds, labels=val_labels)
    print("Validation Metrics:")
    print("Accuracy: ", val_metrics['accuracy'])
    print("Precision: ", val_metrics['precision'])
    print("Recall: ", val_metrics['recall'])
    print("F1 Score: ", val_metrics['f1'])

    # Set the model back to training mode
    model.train()

# Save the model
model.save_pretrained("/content/gdrive/MyDrive/bert_result/epoch_10")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10
Step 0/24, Loss: 0.8599921464920044
Avg Loss for Epoch 1: 0.7765697066982588
Epoch 1 Metrics:
Accuracy:  0.34120734908136485
Precision:  0.618252881433299
Recall:  0.34120734908136485
F1 Score:  0.362531480950502
Epoch completed.
Validation Metrics:
Accuracy:  0.2578125
Precision:  0.6941356169871794
Recall:  0.2578125
F1 Score:  0.27709422183507554
Epoch 2/10
Step 0/24, Loss: 0.8444575667381287
Avg Loss for Epoch 2: 0.7549998884399732
Epoch 2 Metrics:
Accuracy:  0.3963254593175853
Precision:  0.615161741145993
Recall:  0.3963254593175853
F1 Score:  0.4408271081499428
Epoch completed.
Validation Metrics:
Accuracy:  0.71875
Precision:  0.7797327070169605
Recall:  0.71875
F1 Score:  0.7443082524271845
Epoch 3/10
Step 0/24, Loss: 0.6408136487007141
Avg Loss for Epoch 3: 0.6720280547936758
Epoch 3 Metrics:
Accuracy:  0.5905511811023622
Precision:  0.6576771653543306
Recall:  0.5905511811023622
F1 Score:  0.6180775126095897
Epoch completed.
Validation Metrics:
Accuracy:  0.820312

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Metrics:
Accuracy:  0.8515625
Precision:  0.72515869140625
Recall:  0.8515625
F1 Score:  0.783293776371308
Epoch 6/10
Step 0/24, Loss: 0.38142159581184387
Avg Loss for Epoch 6: 0.5298837261895338
Epoch 6 Metrics:
Accuracy:  0.7821522309711286
Precision:  0.6191178609270147
Recall:  0.7821522309711286
F1 Score:  0.6911507195621165
Epoch completed.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Validation Metrics:
Accuracy:  0.8515625
Precision:  0.72515869140625
Recall:  0.8515625
F1 Score:  0.783293776371308
Epoch 9/10
Step 0/24, Loss: 0.35432323813438416
Avg Loss for Epoch 9: 0.5209954480330149
Epoch 9 Metrics:
Accuracy:  0.7874015748031497
Precision:  0.62000124000248
Recall:  0.7874015748031497
F1 Score:  0.6937458808838324
Epoch completed.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Metrics:
Accuracy:  0.8515625
Precision:  0.72515869140625
Recall:  0.8515625
F1 Score:  0.783293776371308
Epoch 10/10
Step 0/24, Loss: 0.43572109937667847
Avg Loss for Epoch 10: 0.5093133306751648
Epoch 10 Metrics:
Accuracy:  0.7874015748031497
Precision:  0.62000124000248
Recall:  0.7874015748031497
F1 Score:  0.6937458808838324
Epoch completed.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Metrics:
Accuracy:  0.8515625
Precision:  0.72515869140625
Recall:  0.8515625
F1 Score:  0.783293776371308


epoch = 15

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Initialize the tokenizer and model for BERT
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")
model = BertForSequenceClassification.from_pretrained("bert-large-uncased")

# The dataset class
class TheDataset(Dataset):
    def __init__(self, reviews, sentiments, tokenizer):
        self.reviews = reviews
        self.sentiments = sentiments
        self.tokenizer = tokenizer
        self.max_len = tokenizer.model_max_length

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, index):
        review = str(self.reviews[index])
        sentiments = self.sentiments[index]

        encoded_review = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
            padding="max_length",
            truncation=True
        )

        return {
            'input_ids': encoded_review['input_ids'][0],
            'attention_mask': encoded_review['attention_mask'][0],
            'labels': torch.tensor(sentiments, dtype=torch.long)
        }

# Prepare the Train/Validation sets
train_dataset = TheDataset(
    reviews=train.LyricsClean.tolist(),
    sentiments=train.sentiment.tolist(),
    tokenizer=tokenizer,
)

val_dataset = TheDataset(
    reviews=val.LyricsClean.tolist(),
    sentiments=val.sentiment.tolist(),
    tokenizer=tokenizer,
)

# Freeze BERT except (the 24th layer + the last pooler layer)
for name, param in model.bert.named_parameters():
    if (not name.startswith('pooler')) and "layer.23" not in name:
        param.requires_grad = False

# Define the function to get the accuracy
def compute_metrics(preds, labels):
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Define the training parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
train_batch_size = 16
eval_batch_size = 64
num_train_epochs = 15
weight_decay = 0.01
warmup_steps = 500
learning_rate = 2e-5

# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=eval_batch_size)

# Create optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
total_steps = len(train_dataloader) * num_train_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

# Start pre-training!
model.train()
for epoch in range(num_train_epochs):
    print(f"Epoch {epoch + 1}/{num_train_epochs}")
    total_loss = 0.0
    total_steps = len(train_dataloader)

    # Initialize variables to store metrics for the epoch
    epoch_preds = []
    epoch_labels = []

    for step, batch in enumerate(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        # Get predictions and labels to compute metrics
        logits = outputs.logits
        preds = logits.argmax(dim=1)
        labels = batch['labels']

        # Store predictions and labels for the epoch
        epoch_preds.extend(preds.detach().cpu().numpy())
        epoch_labels.extend(labels.detach().cpu().numpy())

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        if step % 50 == 0:
            print(f"Step {step}/{total_steps}, Loss: {loss.item()}")

    # Calculate average loss for the epoch
    avg_loss = total_loss / total_steps
    print(f"Avg Loss for Epoch {epoch + 1}: {avg_loss}")

    # Calculate metrics for the epoch
    metrics = compute_metrics(preds=epoch_preds, labels=epoch_labels)
    print(f"Epoch {epoch + 1} Metrics:")
    print("Accuracy: ", metrics['accuracy'])
    print("Precision: ", metrics['precision'])
    print("Recall: ", metrics['recall'])
    print("F1 Score: ", metrics['f1'])

    print("Epoch completed.")

    # Now, let's validate the model after each epoch
    model.eval()
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for val_batch in val_dataloader:
            val_batch = {k: v.to(device) for k, v in val_batch.items()}
            val_outputs = model(**val_batch)
            val_logits = val_outputs.logits
            val_preds.extend(val_logits.argmax(dim=1).detach().cpu().numpy())
            val_labels.extend(val_batch['labels'].detach().cpu().numpy())

    val_metrics = compute_metrics(preds=val_preds, labels=val_labels)
    print("Validation Metrics:")
    print("Accuracy: ", val_metrics['accuracy'])
    print("Precision: ", val_metrics['precision'])
    print("Recall: ", val_metrics['recall'])
    print("F1 Score: ", val_metrics['f1'])

    # Set the model back to training mode
    model.train()

# Save the model
model.save_pretrained("/content/gdrive/MyDrive/bert_result/epoch 15")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/15
Step 0/24, Loss: 1.0479450225830078
Avg Loss for Epoch 1: 1.1277041286230087
Epoch 1 Metrics:
Accuracy:  0.2152230971128609
Precision:  0.8327186075424783
Recall:  0.2152230971128609
F1 Score:  0.07994111213826163
Epoch completed.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Metrics:
Accuracy:  0.1484375
Precision:  0.02203369140625
Recall:  0.1484375
F1 Score:  0.03837159863945579
Epoch 2/15
Step 0/24, Loss: 0.9533699154853821
Avg Loss for Epoch 2: 1.0665076573689778
Epoch 2 Metrics:
Accuracy:  0.2178477690288714
Precision:  0.832838177549706
Recall:  0.2178477690288714
F1 Score:  0.08530077923962355
Epoch completed.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Validation Metrics:
Accuracy:  0.8515625
Precision:  0.72515869140625
Recall:  0.8515625
F1 Score:  0.783293776371308
Epoch 14/15
Step 0/24, Loss: 0.4537166953086853
Avg Loss for Epoch 14: 0.49648387481768924
Epoch 14 Metrics:
Accuracy:  0.7874015748031497
Precision:  0.62000124000248
Recall:  0.7874015748031497
F1 Score:  0.6937458808838324
Epoch completed.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Metrics:
Accuracy:  0.8515625
Precision:  0.72515869140625
Recall:  0.8515625
F1 Score:  0.783293776371308
Epoch 15/15
Step 0/24, Loss: 0.4646741449832916
Avg Loss for Epoch 15: 0.4880116532246272
Epoch 15 Metrics:
Accuracy:  0.7874015748031497
Precision:  0.62000124000248
Recall:  0.7874015748031497
F1 Score:  0.6937458808838324
Epoch completed.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Metrics:
Accuracy:  0.8515625
Precision:  0.72515869140625
Recall:  0.8515625
F1 Score:  0.783293776371308


epoch = 20

In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Initialize the tokenizer and model for BERT
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")
model = BertForSequenceClassification.from_pretrained("bert-large-uncased")

# The dataset class
class TheDataset(Dataset):
    def __init__(self, reviews, sentiments, tokenizer):
        self.reviews = reviews
        self.sentiments = sentiments
        self.tokenizer = tokenizer
        self.max_len = tokenizer.model_max_length

    def __len__(self):
        return len(self.reviews)

    def __getitem__(self, index):
        review = str(self.reviews[index])
        sentiments = self.sentiments[index]

        encoded_review = self.tokenizer.encode_plus(
            review,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            return_attention_mask=True,
            return_tensors='pt',
            padding="max_length",
            truncation=True
        )

        return {
            'input_ids': encoded_review['input_ids'][0],
            'attention_mask': encoded_review['attention_mask'][0],
            'labels': torch.tensor(sentiments, dtype=torch.long)
        }

# Prepare the Train/Validation sets
train_dataset = TheDataset(
    reviews=train.LyricsClean.tolist(),
    sentiments=train.sentiment.tolist(),
    tokenizer=tokenizer,
)

val_dataset = TheDataset(
    reviews=val.LyricsClean.tolist(),
    sentiments=val.sentiment.tolist(),
    tokenizer=tokenizer,
)

# Freeze BERT except (the 24th layer + the last pooler layer)
for name, param in model.bert.named_parameters():
    if (not name.startswith('pooler')) and "layer.23" not in name:
        param.requires_grad = False

# Define the function to get the accuracy
def compute_metrics(preds, labels):
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Define the training parameters
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
train_batch_size = 16
eval_batch_size = 64
num_train_epochs = 20
weight_decay = 0.01
warmup_steps = 500
learning_rate = 2e-5

# Create data loaders
train_dataloader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=eval_batch_size)

# Create optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
total_steps = len(train_dataloader) * num_train_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

# Start pre-training!
model.train()
for epoch in range(num_train_epochs):
    print(f"Epoch {epoch + 1}/{num_train_epochs}")
    total_loss = 0.0
    total_steps = len(train_dataloader)

    # Initialize variables to store metrics for the epoch
    epoch_preds = []
    epoch_labels = []

    for step, batch in enumerate(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        # Get predictions and labels to compute metrics
        logits = outputs.logits
        preds = logits.argmax(dim=1)
        labels = batch['labels']

        # Store predictions and labels for the epoch
        epoch_preds.extend(preds.detach().cpu().numpy())
        epoch_labels.extend(labels.detach().cpu().numpy())

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

        if step % 50 == 0:
            print(f"Step {step}/{total_steps}, Loss: {loss.item()}")

    # Calculate average loss for the epoch
    avg_loss = total_loss / total_steps
    print(f"Avg Loss for Epoch {epoch + 1}: {avg_loss}")

    # Calculate metrics for the epoch
    metrics = compute_metrics(preds=epoch_preds, labels=epoch_labels)
    print(f"Epoch {epoch + 1} Metrics:")
    print("Accuracy: ", metrics['accuracy'])
    print("Precision: ", metrics['precision'])
    print("Recall: ", metrics['recall'])
    print("F1 Score: ", metrics['f1'])

    print("Epoch completed.")

    # Now, let's validate the model after each epoch
    model.eval()
    val_preds = []
    val_labels = []

    with torch.no_grad():
        for val_batch in val_dataloader:
            val_batch = {k: v.to(device) for k, v in val_batch.items()}
            val_outputs = model(**val_batch)
            val_logits = val_outputs.logits
            val_preds.extend(val_logits.argmax(dim=1).detach().cpu().numpy())
            val_labels.extend(val_batch['labels'].detach().cpu().numpy())

    val_metrics = compute_metrics(preds=val_preds, labels=val_labels)
    print("Validation Metrics:")
    print("Accuracy: ", val_metrics['accuracy'])
    print("Precision: ", val_metrics['precision'])
    print("Recall: ", val_metrics['recall'])
    print("F1 Score: ", val_metrics['f1'])

    # Set the model back to training mode
    model.train()

# Save the model
model.save_pretrained("/content/gdrive/MyDrive/bert_result/epoch 20")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/20
Step 0/24, Loss: 0.7759166955947876
Avg Loss for Epoch 1: 0.762155070900917
Epoch 1 Metrics:
Accuracy:  0.3700787401574803
Precision:  0.7173814940456054
Recall:  0.3700787401574803
F1 Score:  0.3747227688480292
Epoch completed.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Metrics:
Accuracy:  0.1484375
Precision:  0.02203369140625
Recall:  0.1484375
F1 Score:  0.03837159863945579
Epoch 2/20
Step 0/24, Loss: 0.7590773105621338
Avg Loss for Epoch 2: 0.7439396927754084
Epoch 2 Metrics:
Accuracy:  0.42782152230971127
Precision:  0.6779376317456116
Recall:  0.42782152230971127
F1 Score:  0.46413216752813874
Epoch completed.
Validation Metrics:
Accuracy:  0.21875
Precision:  0.7969842657342657
Recall:  0.21875
F1 Score:  0.18121936274509803
Epoch 3/20
Step 0/24, Loss: 0.6922047734260559
Avg Loss for Epoch 3: 0.6911280304193497
Epoch 3 Metrics:
Accuracy:  0.5459317585301837
Precision:  0.6707834809397842
Recall:  0.5459317585301837
F1 Score:  0.5863969407803025
Epoch completed.
Validation Metrics:
Accuracy:  0.8203125
Precision:  0.7210811491935484
Recall:  0.8203125
F1 Score:  0.7675026824034334
Epoch 4/20
Step 0/24, Loss: 0.6945123076438904
Avg Loss for Epoch 4: 0.6272533958156904
Epoch 4 Metrics:
Accuracy:  0.6824146981627297
Precision:  0.6690110

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Validation Metrics:
Accuracy:  0.8515625
Precision:  0.72515869140625
Recall:  0.8515625
F1 Score:  0.783293776371308
Epoch 18/20
Step 0/24, Loss: 0.4848194420337677
Avg Loss for Epoch 18: 0.4508000798523426
Epoch 18 Metrics:
Accuracy:  0.7952755905511811
Precision:  0.7792508556751262
Recall:  0.7952755905511811
F1 Score:  0.7204368623735898
Epoch completed.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Metrics:
Accuracy:  0.8515625
Precision:  0.72515869140625
Recall:  0.8515625
F1 Score:  0.783293776371308
Epoch 19/20
Step 0/24, Loss: 0.5382047891616821
Avg Loss for Epoch 19: 0.43955307081341743
Epoch 19 Metrics:
Accuracy:  0.800524934383202
Precision:  0.8117274110789627
Recall:  0.800524934383202
F1 Score:  0.7276051479537543
Epoch completed.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation Metrics:
Accuracy:  0.8515625
Precision:  0.72515869140625
Recall:  0.8515625
F1 Score:  0.783293776371308
Epoch 20/20
Step 0/24, Loss: 0.23693020641803741
Avg Loss for Epoch 20: 0.39814001756409806
Epoch 20 Metrics:
Accuracy:  0.8188976377952756
Precision:  0.8288884694207744
Recall:  0.8188976377952756
F1 Score:  0.7670698786091783
Epoch completed.
Validation Metrics:
Accuracy:  0.828125
Precision:  0.786545868347339
Recall:  0.828125
F1 Score:  0.8012022243107769


## Prediction using Best Checkpoint (epoch 20)

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Load the tokenizer and model configuration
tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")
model = BertForSequenceClassification.from_pretrained("/content/gdrive/MyDrive/bert_result/epoch 20")

# Assuming you already have the test DataFrame loaded as 'test'
test_set_dataset = TheDataset(
    reviews=test.LyricsClean.tolist(),
    sentiments=test.sentiment.tolist(),
    tokenizer=tokenizer,
)

test_dataloader = DataLoader(test_set_dataset, batch_size=1)  # Set batch_size=1 for inference

# Put the model in evaluation mode
model.eval()

# Perform prediction on the test set
predictions = []
with torch.no_grad():
    for batch in test_dataloader:
        inputs = {
            'input_ids': batch['input_ids'],
            'attention_mask': batch['attention_mask']
        }
        outputs = model(**inputs)
        predicted_class = outputs.logits.argmax(dim=-1)
        predictions.append(predicted_class.item())

# Add the predicted labels to the test DataFrame
test["predicted_sentiment"] = predictions

# Compute precision, recall, F1 score, and accuracy
labels = test.sentiment.tolist()
predicted_labels = test.predicted_sentiment.tolist()
precision, recall, f1, _ = precision_recall_fscore_support(labels, predicted_labels, average='binary')
accuracy = accuracy_score(labels, predicted_labels)

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("Accuracy:", accuracy)

# Display the test DataFrame with the added predicted labels
print(test)


Precision: 0.775
Recall: 0.9893617021276596
F1 Score: 0.869158878504673
Accuracy: 0.78125
                                                                                  lyrics  \
title                                                                                      
Wait A Minute                          Why is it so hard? Youre filled with mystery I...   
IDOL                                   You can call me artist (artist)\nYou can call ...   
Attack on Bangtan/The Rise of Bangtan  but what will happen if bangtan seonyeondan ri...   
I Like It                              wanna be loveda|\n\ndonat wanna be fool wanna ...   
Let me tell you right now              When it's quiet, I still look for you Sighing ...   
...                                                                                  ...   
So What                                somebody call me right one\nsomebody call me w...   
Paradise                               marathon marathon\nlifeas long so take it s

In [ ]:
pip install accelerate>=0.20.1


In [ ]:
pip install transformers[torch]
